In [2]:
import imageio.v3 as iio
import matplotlib.pyplot as plt
import requests
import numpy as np
%matplotlib inline


In [3]:
api_url = "https://www.flickr.com/services/rest/?method=flickr.photos.search&api_key=f8ac40eee0aacfa3be6dee0cbac63930" +\
            "&tags=sunset" +\
            "&text=sunset%20landscape" +\
            "&content_type=1" +\
            "&has_geo=1" +\
            "&sort=interestingness-desc" +\
            "&extras=date_taken%2C+geo%2C+url_o" +\
            "&format=json&nojsoncallback=1"
            # "&is_getty=1" +\             

res = requests.get(api_url)
res = res.json()

In [4]:
# look at later for batch processing
'''
fig = plt.figure(figsize=(10, 5))
rows = 2
cols = 5

for i in range(10):
    # get and load pic
    photo = res['photos']['photo'][i]
    print(photo['title'])
    # print(photo["url_o"])
    imgUrl = f'https://live.staticflickr.com/{photo["server"]}/{photo["id"]}_{photo["secret"]}_b.jpg'
    aPic = iio.imread(imgUrl)

    # add aPic to fig
    fig.add_subplot(rows, cols, i+1)
    plt.imshow(pic) 
'''

'\nfig = plt.figure(figsize=(10, 5))\nrows = 2\ncols = 5\n\nfor i in range(10):\n    # get and load pic\n    photo = res[\'photos\'][\'photo\'][i]\n    print(photo[\'title\'])\n    # print(photo["url_o"])\n    imgUrl = f\'https://live.staticflickr.com/{photo["server"]}/{photo["id"]}_{photo["secret"]}_b.jpg\'\n    aPic = iio.imread(imgUrl)\n\n    # add aPic to fig\n    fig.add_subplot(rows, cols, i+1)\n    plt.imshow(pic) \n'

In [5]:
aPhoto = res['photos']['photo'][0]
aImgUrl = f'https://live.staticflickr.com/{aPhoto["server"]}/{aPhoto["id"]}_{aPhoto["secret"]}_b.jpg'
aFig = plt.figure(figsize=(10, 10))
aPic = iio.imread(aImgUrl)

<Figure size 720x720 with 0 Axes>

In [7]:
# random.seed(1)
print('Shape of the image : {}'.format(aPic[:,:,0].shape)) 
# print('Type of the image : ' , type(pic))
# print('Shape of the image : {}'.format(pic.shape)) 
# print('Image Hight {}'.format(pic.shape[0])) 
# print('Image Width {}'.format(pic.shape[1])) 
# print('Dimension of Image {}'.format(pic.ndim))


# get random selection of pixels from image to analyze
indices = np.random.choice(aPic.shape[1]*aPic.shape[0], replace=False, size=int(aPic.shape[1]*aPic.shape[0]*.25))
print(indices)
print(indices.size)
print(aPic[:,:,0].shape)
print(np.unravel_index(indices, aPic[:,:,0].shape)) # how to unravel randomized indexes?
# sampledPic = aPic[np.unravel_index(indices, aPic[:,:,0].shape), :]
# print(sampledPic.shape)
# aPic[1,1].shape
# get amount of pixels with certain hue | amount of red 

Shape of the image : (613, 1024)
[165768 155438 449681 ... 430430 181201 570386]
156928
(613, 1024)
(array([161, 151, 439, ..., 420, 176, 557], dtype=int64), array([904, 814, 145, ..., 350, 977,  18], dtype=int64))
